In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
paragraphs_gold = pd.read_csv('./all_paragraphs.csv') 
paragraphs_gold

,text-id,text,label,version
0,medium-17,"Due to the overreach on the initial layoffs, c...",1,3
1,medium-17,I used to be in Pharmacy. Then I did a researc...,0,3
2,medium-17,So if a medical center has any that are partia...,1,3
3,medium-17,Open vials are supposed to be discarded after ...,1,3
4,medium-17,It costs about 15-35 cents to make a mL of ins...,1,3
...,...,...,...,...
166,hard-252,We don’t make more money today as land develop...,0,-1
167,hard-252,"Blame red tape, environmental “protection”, an...",1,-1
168,hard-252,If you want the government to intervene to mak...,1,-1
169,hard-252,Most politicians in my province (and probably ...,0,-1


In [3]:
dataframes = []
for i in range(1,9):
    dataframes.append(pd.read_csv(f'./paragraphs_labels/ver_{i}.csv'))

def parse_array_string(s):
    # Use regex to find all occurrences of numbers inside array([...])
    array_strings = re.findall(r'\[([0-9, ]+)\]', s)
    # Convert found strings to numpy arrays
    arrays = [np.fromstring(arr, sep=', ', dtype=int) for arr in array_strings]
    return arrays

for j in range(len(dataframes)):
    df = dataframes[j]
    for i in range(len(df)):
        s = df['annotator_paragraph'][i]
        # Extract the numpy arrays from the string
        arrays = parse_array_string(s)

        # Convert to a numpy array of numpy arrays
        result = arrays
        df['annotator_paragraph'][i] = result


In [4]:
dataframes[0]['annotator_paragraph'][0]

[array([1, 1, 1, 1, 1, 0, 1, 1, 0, 1]),
 array([0, 1, 1, 0, 1, 1, 1, 1, 1, 1]),
 array([1, 1, 1, 1, 1, 0, 1, 0, 0, 0])]

In [5]:
df1 = dataframes[0]
df2 = paragraphs_gold

# Create a dictionary to map text-id to paragraphs
paragraph_dict = df1.groupby('text-id')['annotator_paragraph'].apply(list).to_dict()
for key, value in paragraph_dict.items():
    paragraph_dict[key] = value[0]

# Initialize an empty list to store the merged rows
merged_rows = []

for index, row in df2.iterrows():
    text_id = row['text-id']
    paragraphs = paragraph_dict.get(text_id, [])
    if paragraphs:
        paragraph = paragraphs[index % len(paragraphs)]
        row['annotator_paragraph'] = paragraph
    
    merged_rows.append(row)

merged_df = pd.DataFrame(merged_rows)

In [6]:
merged_df.loc[merged_df['text-id'] == 'medium-534']

,text-id,text,label,version,annotator_paragraph
57,medium-534,Likely illegal. The City of Seattle now runs a...,1,1,"[1, 1, 1, 1, 1, 0, 1, 1, 0, 1]"
58,medium-534,Did the landlord put that in writing? It is ve...,1,1,"[0, 1, 1, 0, 1, 1, 1, 1, 1, 1]"
59,medium-534,"If illegal, you may need a local real estate l...",1,1,"[1, 1, 1, 1, 1, 0, 1, 0, 0, 0]"


In [7]:
def merge_gold_annotator(df1, df2):
    # Create a dictionary to map text-id to paragraphs
    paragraph_dict = df1.groupby('text-id')['annotator_paragraph'].apply(list).to_dict()
    for key, value in paragraph_dict.items():
        paragraph_dict[key] = value[0]

    # Initialize an empty list to store the merged rows
    merged_rows = []

    for index, row in df2.iterrows():
        text_id = row['text-id']
        paragraphs = paragraph_dict.get(text_id, [])
        if paragraphs:
            paragraph = paragraphs[index % len(paragraphs)]
            row['annotator_paragraph'] = paragraph
        
        merged_rows.append(row)

    merged_df = pd.DataFrame(merged_rows)

    return merged_df


for i in range(len(dataframes)):
    paragraphs_gold = merge_gold_annotator(dataframes[i], paragraphs_gold)

In [8]:
paragraphs_gold

,text-id,text,label,version,annotator_paragraph
0,medium-17,"Due to the overreach on the initial layoffs, c...",1,3,"[1, 1, 1, 1, 1, 1, 1, 0, 1]"
1,medium-17,I used to be in Pharmacy. Then I did a researc...,0,3,"[1, 0, 0, 0, 1, 1, 0, 0, 0]"
2,medium-17,So if a medical center has any that are partia...,1,3,"[0, 0, 0, 0, 0, 0, 1, 0, 0]"
3,medium-17,Open vials are supposed to be discarded after ...,1,3,"[1, 0, 0, 1, 1, 0, 1, 0, 0]"
4,medium-17,It costs about 15-35 cents to make a mL of ins...,1,3,"[1, 0, 0, 0, 0, 1, 0, 1, 1]"
...,...,...,...,...,...
166,hard-252,We don’t make more money today as land develop...,0,-1,NaN
167,hard-252,"Blame red tape, environmental “protection”, an...",1,-1,NaN
168,hard-252,If you want the government to intervene to mak...,1,-1,NaN
169,hard-252,Most politicians in my province (and probably ...,0,-1,NaN


In [9]:
paragraphs_gold.loc[paragraphs_gold['text-id'] == 'medium-17']

,text-id,text,label,version,annotator_paragraph
0,medium-17,"Due to the overreach on the initial layoffs, c...",1,3,"[1, 1, 1, 1, 1, 1, 1, 0, 1]"
1,medium-17,I used to be in Pharmacy. Then I did a researc...,0,3,"[1, 0, 0, 0, 1, 1, 0, 0, 0]"
2,medium-17,So if a medical center has any that are partia...,1,3,"[0, 0, 0, 0, 0, 0, 1, 0, 0]"
3,medium-17,Open vials are supposed to be discarded after ...,1,3,"[1, 0, 0, 1, 1, 0, 1, 0, 0]"
4,medium-17,It costs about 15-35 cents to make a mL of ins...,1,3,"[1, 0, 0, 0, 0, 1, 0, 1, 1]"
5,medium-17,The Canadian team who isolated and purified in...,1,3,"[1, 1, 1, 1, 1, 0, 0, 0, 1]"
6,medium-17,"Medical waste is ridiculous, and definitely ha...",1,3,"[1, 1, 0, 1, 1, 0, 1, 0, 1]"


In [10]:
paragraphs_gold.loc[paragraphs_gold['annotator_paragraph'].isna()]

,text-id,text,label,version,annotator_paragraph
149,hard-784,If the west stops dragging their feet and arms...,0,3,NaN
150,hard-784,Russia can be defeated in 2023 if the West sto...,1,3,NaN
151,hard-784,"if Ukraine is armed to the teeth, we ought to ...",1,3,NaN
152,hard-784,Depends if he uses tactical or strategic nukes...,1,3,NaN
153,hard-784,"It's literally been stated by NATO, nuclear fa...",1,3,NaN
154,hard-784,Even if russia nukes Ukraine with a strategic ...,1,3,NaN
155,hard-784,But using any nukes at all in this fashion wou...,1,3,NaN
166,hard-252,We don’t make more money today as land develop...,0,-1,NaN
167,hard-252,"Blame red tape, environmental “protection”, an...",1,-1,NaN
168,hard-252,If you want the government to intervene to mak...,1,-1,NaN


In [21]:
paragraphs_gold.loc[paragraphs_gold['text-id'] == 'medium-277']
paragraphs_new = paragraphs_gold.drop(index=[38, 39])

In [28]:
paragraphs_new.loc[paragraphs_gold['text-id'] == 'medium-277']

,text-id,text,label,version,annotator_paragraph
40,medium-277,"Most bank statements, apps, etc. will have pho...",1,7,"[0, 0, 0, 1, 0, 0, 1]"
41,medium-277,When you can not get that form filled out (for...,1,7,"[0, 0, 1, 1, 0, 0, 1]"
42,medium-277,If your handwritten checks do not show how muc...,0,7,"[0, 1, 1, 0, 1, 0, 0]"


In [40]:
paragraphs_new.loc[paragraphs_new['text-id'] == 'medium-277']

,text-id,text,label,version,annotator_paragraph
40,medium-277,"Most bank statements, apps, etc. will have pho...",1,7,"[0, 0, 0, 1, 0, 0, 1]"
41,medium-277,When you can not get that form filled out (for...,1,7,"[0, 0, 1, 1, 0, 0, 1]"
42,medium-277,If your handwritten checks do not show how muc...,0,7,"[0, 1, 1, 0, 1, 0, 0]"


In [35]:
dataframes[6]['annotator_paragraph'][1][1:]

[array([0, 0, 1, 1, 0, 0, 1]),
 array([0, 1, 1, 0, 1, 0, 0]),
 array([0, 0, 0, 0, 1, 0, 0])]

In [49]:
paragraphs_new.to_csv('paragraph_gold_annotator.csv', index=False)